<a href="https://colab.research.google.com/github/Daffaaq/BIG_DATA_SPARK_ML_Naive-Bayes/blob/main/Spark_Machine-Learning_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NAMA Kelompok: 


1.   Aida Millati Mardiana (2041720071)
2.   Daffa Aqila Rahmatullah (2041720098)
3.   Dilta Febiana (2041720136)



### Persiapan

In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz

!tar xf spark-3.4.0-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
pip install pyspark==2.4.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.7/215.7 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.3/197.3 kB 18.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130372 sha256=848f1ed84934f20e74e39aeab784499e3cbddbad78e7f8126166505334134ef2
  Stored in directory: /root/.cache/pip/wheels/d1/bb/c7/1323feaa6ff889a2471f9c82d07a83bd2ce52e8fb12f86a45a
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
import os
os.environ["JAVA_HOME"] = "/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

In [3]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate();

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Organizing data in DataFrames

In [5]:
irisTxtPath = '/content/drive/MyDrive/Semester 5 ML/Pertemuan 8/iris.csv'

In [6]:
empDF = spark.read.csv(irisTxtPath, header=True)

In [7]:
empDF

DataFrame[5.1: string, 3.5: string, 1.4: string, 0.2: string, Iris-setosa: string]

In [8]:
empDF.schema

StructType([StructField('5.1', StringType(), True), StructField('3.5', StringType(), True), StructField('1.4', StringType(), True), StructField('0.2', StringType(), True), StructField('Iris-setosa', StringType(), True)])

In [9]:
empDF.printSchema()

root
 |-- 5.1: string (nullable = true)
 |-- 3.5: string (nullable = true)
 |-- 1.4: string (nullable = true)
 |-- 0.2: string (nullable = true)
 |-- Iris-setosa: string (nullable = true)



In [10]:
empDF.columns

['5.1', '3.5', '1.4', '0.2', 'Iris-setosa']

In [11]:
empDF.take(5)

[Row(5.1='4.9', 3.5='3.0', 1.4='1.4', 0.2='0.2', Iris-setosa='Iris-setosa'),
 Row(5.1='4.7', 3.5='3.2', 1.4='1.3', 0.2='0.2', Iris-setosa='Iris-setosa'),
 Row(5.1='4.6', 3.5='3.1', 1.4='1.5', 0.2='0.2', Iris-setosa='Iris-setosa'),
 Row(5.1='5.0', 3.5='3.6', 1.4='1.4', 0.2='0.2', Iris-setosa='Iris-setosa'),
 Row(5.1='5.4', 3.5='3.9', 1.4='1.7', 0.2='0.4', Iris-setosa='Iris-setosa')]

In [12]:
empDF.count()

149

In [13]:
sampleDF = empDF.sample(False, 0.1)
sampleDF.count()

10

In [15]:
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

In [16]:
irisDF = spark.read.csv(irisTxtPath, inferSchema=True)

In [17]:
# get new iris txt file to read into for example
irisDF.columns

['_c0', '_c1', '_c2', '_c3', '_c4']

In [18]:
irisDF = irisDF.select(col('_c0').alias('sepal_length'),
                       col('_c1').alias('sepal_width'),
                       col('_c2').alias('petal_length'),
                       col('_c3').alias('petal_width'),
                       col('_c4').alias('species'))

In [19]:
irisDF.take(1)

[Row(sepal_length=5.1, sepal_width=3.5, petal_length=1.4, petal_width=0.2, species='Iris-setosa')]

In [20]:
irisDF.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- species: string (nullable = true)



In [21]:
vectorAssembler = VectorAssembler(inputCols=['sepal_length',
                                             'sepal_width', 
                                             'petal_length',
                                             'petal_width'],
                                  outputCol='features')

In [22]:
virisDF = vectorAssembler.transform(irisDF)
virisDF.take(1)

[Row(sepal_length=5.1, sepal_width=3.5, petal_length=1.4, petal_width=0.2, species='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]))]

In [23]:
indexer = StringIndexer(inputCol='species', outputCol='label')
iVirisDF = indexer.fit(virisDF).transform(virisDF)
iVirisDF

DataFrame[sepal_length: double, sepal_width: double, petal_length: double, petal_width: double, species: string, features: vector, label: double]

### Naive Bayes Classification

In [24]:
iVirisDF.take(1)

[Row(sepal_length=5.1, sepal_width=3.5, petal_length=1.4, petal_width=0.2, species='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]), label=0.0)]

In [25]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [26]:
splits = iVirisDF.randomSplit([0.6,0.4],1)
trainDF = splits[0]
testDF = splits[1]

In [27]:
trainDF.count()

98

In [28]:
testDF.count()

52

In [29]:
iVirisDF.count()

150

In [30]:
nb = NaiveBayes(modelType='multinomial')

In [31]:
nbModel = nb.fit(trainDF)

In [32]:
predictionsDF = nbModel.transform(testDF)

In [33]:
predictionsDF.take(1)

[Row(sepal_length=4.3, sepal_width=3.0, petal_length=1.1, petal_width=0.1, species='Iris-setosa', features=DenseVector([4.3, 3.0, 1.1, 0.1]), label=0.0, rawPrediction=DenseVector([-9.9894, -11.3476, -11.902]), probability=DenseVector([0.7118, 0.183, 0.1051]), prediction=0.0)]

In [34]:
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')

In [35]:
nbAccuracy = evaluator.evaluate(predictionsDF)
nbAccuracy

0.9807692307692307